# PA1: Tracking with the α–β (g-h) Filter  

PA1 covers intro to tracking.  

The students implement an $\alpha$-$\beta$ tracker to maxmimize the sensor accuracy and discuss its limitations.




## Learning Goals
By the end of this assignment, you will be able to:
- Implement an α–β (g-h) filter from scratch.  
- Vectorize Python code for efficiency.  
- Evaluate filter performance using Mean Squared Error (MSE).  
- Tune g and h parameters and explain the tradeoffs between responsiveness and noise suppression.  
- Reflect on the strengths and limitations of the α–β filter.  

In [ ]:
import numpy as np
rng = np.random.RandomState(42)

In [ ]:
from __future__ import division, print_function
%matplotlib inline

In [ ]:
# # Colab makeup

# Clone the book repo
!git clone https://github.com/rlabbe/Kalman-and-Bayesian-Filters-in-Python.git
# Change into the repo directory
%cd Kalman-and-Bayesian-Filters-in-Python
# Install requirements (optional, but good to have)
!pip install -r requirements.txt
# Add repo to Python path
import sys
sys.path.append('/content/Kalman-and-Bayesian-Filters-in-Python')
print("Finished setup!")

Cloning into 'Kalman-and-Bayesian-Filters-in-Python'...
remote: Enumerating objects: 7036, done.
remote: Counting objects: 100% (52/52), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 7036 (delta 35), reused 25 (delta 25), pack-reused 6984 (from 3)
Receiving objects: 100% (7036/7036), 585.24 MiB | 36.28 MiB/s, done.
Resolving deltas: 100% (3853/3853), done.
/content/Kalman-and-Bayesian-Filters-in-Python
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 65.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.7/76.7 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 63.8 MB/s eta 0:00:00
  Created wheel for filterpy: filename=filterpy-1.4.5-py3-none-any.whl size=110460 sha256=235582a3a93dc94d86b2cb78f043aa7201c26082841590b

In [ ]:
# # Colab makeup
#format the book
import runpy
runpy.run_path('book_format.py')

/content/Kalman-and-Bayesian-Filters-in-Python/kf_book/book_plots.py:139: SyntaxWarning: invalid escape sequence '\m'
  ax.annotate('Measurement ($\mathbf{z_k}$)',
/content/Kalman-and-Bayesian-Filters-in-Python/kf_book/book_plots.py:154: SyntaxWarning: invalid escape sequence '\m'
  ax.annotate('Initial\nConditions ($\mathbf{x_0}$)',
/content/Kalman-and-Bayesian-Filters-in-Python/kf_book/book_plots.py:161: SyntaxWarning: invalid escape sequence '\m'
  plt.text (4, 3.7,'State Estimate ($\mathbf{\hat{x}_k}$)',


{'__name__': '<run_path>',
 '__doc__': 'Copyright 2015 Roger R Labbe Jr.\n\n\nCode supporting the book\n\nKalman and Bayesian Filters in Python\nhttps://github.com/rlabbe/Kalman-and-Bayesian-Filters-in-Python\n\n\nThis is licensed under an MIT license. See the LICENSE.txt file\nfor more information.\n',
 '__package__': '',
 '__loader__': None,
 '__spec__': None,
 '__file__': 'book_format.py',
 '__cached__': None,
 '__builtins__': {'__name__': 'builtins',
  '__doc__': "Built-in functions, types, exceptions, and other objects.\n\nThis module provides direct access to all 'built-in'\nidentifiers of Python; for example, builtins.len is\nthe full name for the built-in function len().\n\nThis module is not normally accessed explicitly by most\napplications, but can be useful in modules that provide\nobjects with the same name as a built-in value, but in\nwhich the built-in of that name is also needed.",
  '__package__': '',
  '__loader__': _frozen_importlib.BuiltinImporter,
  '__spec__': Mod

In [ ]:
import kf_book.gh_internal as gh
import kf_book.book_plots as book_plots

from kf_book.book_plots import figsize, plot_errorbars
import matplotlib.pyplot as plt

from kf_book.gh_internal import plot_g_h_results
import matplotlib.pylab as pylab


/content/Kalman-and-Bayesian-Filters-in-Python/kf_book/gh_internal.py:140: SyntaxWarning: invalid escape sequence '\h'
  plt.text (0.0, 159.8, "last estimate ($\hat{x}_{t-1}$)", ha='left', va='top',fontsize=18)
/content/Kalman-and-Bayesian-Filters-in-Python/kf_book/gh_internal.py:166: SyntaxWarning: invalid escape sequence '\h'
  plt.text (0, 159.8, "last estimate ($\hat{x}_{t-1}$)", ha='left', va='top',fontsize=18)
/content/Kalman-and-Bayesian-Filters-in-Python/kf_book/gh_internal.py:167: SyntaxWarning: invalid escape sequence '\h'
  plt.text (0.95, est_y, "estimate ($\hat{x}_{t}$)", ha='right', va='center',fontsize=18)


First, we will vectorize some of the code shared in the lecture.

## Exercise: create measurement function

### Part A. Building the Filter  

1. Vectorize some of the tracking code shown in lecture.  
   - Replace `for` loops with array-based operations.  
   - Confirm your version gives the same results.  

2. Implement a measurement function.  
   - This function should simulate noisy sensor measurements.  

Below is the gen_data_for method that we developed to create noisy data.

In [ ]:
rng = np.random.RandomState(42)

def gen_data_for(x0, dx, count, noise_factor):
    '''
    Generates noisy data
    In:
        'x0' is the initial value for our state variable
        'dx' is the change rate for our state variable
        'count' is the number of steps
        'noise_factor' is the amount of noise we want to add
    Out:
        'measurements' np array, length = count
    '''
    return [x0 + dx*i + rng.randn()*noise_factor for i in range(count)]

The code above returns the desired results. However, it is not the most efficient implementation.

Create another version of this method that does not use a for loop and takes advantage of vectorization.

Your **(solution hidden in student version)** does not have to in one-line. It just needs to be vectorized (i.e. no `for` loops).

Use `rng.randn()` in your code (instead of `np.random.randn()`, to make that you are seeding your number generator as expected.

Hint: Utilize `np.arange()`, `np.linspace()` or similar.

In [ ]:
rng = np.random.RandomState(42)

def gen_data(x0, dx, count, noise_factor):
    '''
    Generates noisy data
    In:
        'x0' is the initial value for our state variable
        'dx' is the change rate for our state variable
        'count' is the number of steps
        'noise_factor' is the amount of noise we want to add
    Out:
        'measurements' np array, length = count
    '''
    ### YOUR CODE HERE
  raise NotImplementedError()

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 15)

In [ ]:
#user test area
np.set_printoptions(precision=4)

x0=0; dx=1; count=30; noise_factor=1
rng = np.random.RandomState(42)
measurements_for = gen_data_for(x0, dx, count, noise_factor)
rng = np.random.RandomState(42)
measurements = gen_data(x0,dx, count, noise_factor)

print(measurements_for)
print(measurements)

In [ ]:
x0=0; dx=1; count=5; noise_factor=1
rng = np.random.RandomState(42)
measurements_for = gen_data_for(x0, dx, count, noise_factor)
rng = np.random.RandomState(42)
measurements = gen_data(x0,dx, count, noise_factor)
np.testing.assert_allclose( measurements_for, measurements)



In [ ]:
x0=0; dx=1; count=30; noise_factor=1
rng = np.random.RandomState(42)
measurements_for = gen_data_for(x0, dx, count, noise_factor)
rng = np.random.RandomState(42)
measurements = gen_data(x0,dx, count, noise_factor)
np.testing.assert_allclose( measurements_for, measurements)



In [ ]:
x0=5; dx=2; count=30; noise_factor=3
rng = np.random.RandomState(42)
measurements_for = gen_data_for(x0, dx, count, noise_factor)
rng = np.random.RandomState(42)
measurements = gen_data(x0,dx, count, noise_factor)
np.testing.assert_allclose( measurements_for, measurements)



## Exercise: The Effect of Acceleration

Below is the gen_data_for method that we developed to create noisy data.

In [ ]:
rng = np.random.RandomState(42)

def gen_data_accel_for(x0, dx, count, noise_factor, accel=0.):
    '''
    Generates noisy data
    In:
        'x0' is the initial value for our state variable
        'dx' is the change rate for our state variable
        'count' is the number of steps
        'noise_factor' is the amount of noise we want to add
        'accel' is the constant acceleration factor added to each data point
    Out:
        'measurements' np array, length = count
    '''
    zs = []
    for i in range(count):
        zs.append(x0 + dx*i + rng.randn()*noise_factor)
        dx += accel
    return zs


The code above returns the desired results. However, it is not the most efficient implementation.

Create another version of this method that does not use a for loop and takes advantage of vectorization.

Your **(solution hidden in student version)** does not have to in one-line. It just needs to be vectorized (i.e. no `for` loops).

Use `rng.randn()` in your code (instead of `np.random.randn()`, to make that you are seeding your number generator as expected.

Hint: Utilize `np.arange()`, `np.linspace()` or similar.

In [ ]:
rng = np.random.RandomState(42)

def gen_data_accel(x0, dx, count, noise_factor, accel=0.):
    '''
    Generates noisy data
    In:
        'x0' is the initial value for our state variable
        'dx' is the change rate for our state variable
        'count' is the number of steps
        'noise_factor' is the amount of noise we want to add
        'accel' is the constant acceleration factor added to each data point
    Out:
        'measurements' np array, length = count
    '''
    ### YOUR CODE HERE
  raise NotImplementedError()

In [ ]:
#user test area
np.set_printoptions(precision=4)

x0=0; dx=1; count=30; noise_factor=1; accel=0.;
rng = np.random.RandomState(42)
measurements_for = gen_data_accel_for(x0, dx, count, noise_factor, accel)
rng = np.random.RandomState(42)
measurements = gen_data_accel(x0,dx, count, noise_factor, accel)

print(measurements_for)
print(measurements)

In [ ]:
x0=0; dx=1; count=5; noise_factor=1;accel=0.;
rng = np.random.RandomState(42)
measurements_for = gen_data_accel_for(x0, dx, count, noise_factor, accel)
rng = np.random.RandomState(42)
measurements = gen_data_accel(x0,dx, count, noise_factor, accel)
np.testing.assert_allclose( measurements_for, measurements)



In [ ]:
x0=0; dx=1; count=30; noise_factor=1;accel=1.;
rng = np.random.RandomState(42)
measurements_for = gen_data_accel_for(x0, dx, count, noise_factor, accel)
rng = np.random.RandomState(42)
measurements = gen_data_accel(x0,dx, count, noise_factor, accel)
np.testing.assert_allclose( measurements_for, measurements)



In [ ]:
x0=5; dx=2; count=30; noise_factor=3;accel=2.;
rng = np.random.RandomState(42)
measurements_for = gen_data_accel_for(x0, dx, count, noise_factor, accel)
rng = np.random.RandomState(42)
measurements = gen_data_accel(x0,dx, count, noise_factor, accel)
np.testing.assert_allclose( measurements_for, measurements)



## Filtering: Error calculation


If the true data is known, we can evaluate the performance of our filter.

The code below uses the filter from the lecture against simulated data to estimate its performance.

In [ ]:
rng = np.random.RandomState(42)
#true positions of
positions = np.array([10, 20, 30, 40, 50])
#measured/noisy positions
positions_n = positions+rng.randn(5)

distances = np.array([10, 10, 10, 10, 10])
#measured/noisy positions
distances_n = distances+rng.randn(5)


In [ ]:
from kf_book.gh_internal import plot_g_h_results
import matplotlib.pylab as pylab

def g_h_filter(data, x0, dx, g, h, dt=1.):
    '''
    Performs g-h filter on 1 state variable with a fixed g and h.
    In:
        'data' contains the data to be filtered.
        'x0' is the initial value for our state variable
        'dx' is the initial change rate for our state variable
        'g' is the g-h's g scale factor
        'h' is the g-h's h scale factor
        'dt' is the length of the time step
    Out:
        'results' np array, same shape as data
    '''
    x_est = x0
    results = []
    for z in data:
        # prediction step
        x_pred = x_est + (dx*dt)
        dx = dx

        # update step
        residual = z - x_pred
        dx = dx + h * (residual) / dt
        x_est = x_pred + g * residual
        results.append(x_est)
    return np.array(results)

Code below calls the filter with given data using some conditions:

In [ ]:
#test code here
data = positions_n
estimates = g_h_filter(data=data, x0=10., dx=1., g=6./10, h=2./3, dt=1.)
plot_g_h_results(data, estimates)
estimates

## Part B. Evaluating the Filter  

1. Implement a method that calculates the **Mean Squared Error (MSE):**  

   $$
   \operatorname{MSE} = \frac{1}{n}\sum_{i=1}^n (x_i - \hat{x}_i)^2
   $$  

   - Make sure your implementation is **vectorized** (no Python loops).  
   - Verify it works with simple test inputs.  


Vectorize your code (i.e. no `for` loops).

In [ ]:
def mse(truth, estimates):
    '''
    Calculate the mean squared error (MSE)
    In:
        'truth' contains the unnoisy/true data.
        'estiamtes' contains the g_h_filter's estimate of noisy data.
    Out:
        'mse' shape = 1
    '''
    ### YOUR CODE HERE
  raise NotImplementedError()


In [ ]:
#test your code here
data = positions_n; true=positions;
estimates = g_h_filter(data=data, x0=10., dx=1., g=6./10, h=2./3, dt=1.)
#filter error
print(mse(true, estimates))
# 4.27245219607
#sensor error
print(mse(true, data))
# 0.611958031114


In [ ]:
"""Check that method returns the correct output for several inputs"""
data = positions_n; true=positions;
estimates = g_h_filter(data=data, x0=10., dx=1., g=6./10, h=2./3, dt=1.)
np.testing.assert_allclose( mse(true, estimates), 4.27245219607)

np.testing.assert_allclose( mse(true, data), 0.611958031114)


In [ ]:
"""Check that method returns the correct output for several inputs"""
data = distances_n; true=distances;
estimates = g_h_filter(data=data, x0=10., dx=1., g=6./10, h=2./3, dt=1.)
np.testing.assert_allclose( mse(true, estimates), 0.609509847759)

np.testing.assert_allclose( mse(true, data), 0.730493378945)


In [ ]:
"""Check that function raises an error for invalid inputs"""
#verify input sizes
data = positions_n
estimates = g_h_filter(data=data[3:], x0=10., dx=1., g=6./10, h=2./3, dt=1.)

try:
    mse(data, estimates)
except ValueError:
    pass
else:
    raise AssertionError("Data sizes must match.")

## Part C. Tuning the Filter  

Notice that our filter's estimated position is worse than the measured one.

This defeats the purpose of using a filter.

Tune the filter to optimize its performance.

1. Use the provided data simulating a ball’s position captured by a camera.  
2. Experiment with different `(g, h)` values:  
   - Start with a “bad” guess.  
   - Adjust until the filter tracks well.  
3. Use the provided `tuner()` and `plot_g_h_results()` to visualize results.  

 **Your Task:**  
- Record your chosen `(g, h)` values.  
- Explain in words why your choice balances responsiveness and noise reduction.  
- Before running, make a **prediction**: What will happen if g is very high but h is very small? Did your experiment confirm it?  

Write a utility function named `g_h_tuner()` that given the noisy and truth data, it will return the `g` and `h` vaules that will minimize the filter's error.

`g` and `h` will be limited to [0, 1] and have precision to 2 decimal places (i.e: 0.45).

In [ ]:

def g_h_tuner(truth, data, x0, dx):
    '''
    Calculate the mean squared error (MSE)
    In:
        'truth' contains the unnoisy/true data.
        'data' contains the noisy data.
        'x0' is the initial value for our state variable
        'dx' is the initial change rate for our state variable

    Out:
        'g', 'h': filter paramteres that min fimlter error.
                They exist in [0, 1] and have a precision limited to 2 decimal places
    '''
    ### YOUR CODE HERE
  raise NotImplementedError()

In [ ]:
#test your code here
data = positions_n; true=positions;
x0=10.; dx=1.
g,h = g_h_tuner(true, data, x0, dx)
# (0.94, 0.04)
# mse = 0.39

In [ ]:
#visualize your results
est = g_h_filter(data, x0=x0, dx=dx, g=g, h=h, dt=1/100.)
plot_g_h_results(est, data)

In [ ]:
"""Check that method returns the correct output for several inputs"""

# mse = 0.39
data = positions_n; true=positions;
x0=10.; dx=1.
g,h = g_h_tuner(true, data, x0, dx)
np.testing.assert_allclose([g,h], [0.94, 0.04], rtol=0.01)

In [ ]:
#mse = 0.30
x0=1.; dx=5.
g,h = g_h_tuner(true, data, x0, dx)
np.testing.assert_allclose([g,h], [0.88, 0.08], rtol=0.01)

### BEGIN HIDDEN TESTS
#mse = 0.0
x0=0.; dx=10.
g,h = g_h_tuner(true, data, x0, dx)
np.testing.assert_allclose([g,h], [0., 0.], rtol=0.01)
### END HIDDEN TESTS

You can modify the initial conditions above to reduce the mse to 0.

What do you think these conditions would be? What would the associated `g` and `h` values be?

## Exercise: Heatmap

Create a function that produces a heatmap showing how MSE varies with different g and h parameters

In [ ]:
def g_h_mse_heatmap(truth, data, x0, dx, g_range=(0,1), h_range=(0,1), resolution=11):
    '''
    Creates a heatmap showing how MSE varies with different g and h values.

    Input:
    - truth: ground truth data
    - data: measurement data
    - x0, dx: initial state
    - g_range, h_range: parameter ranges to explore
    - resolution: number of points to sample in each dimension

    Output:
    - Heatmap visualization
    - The g, h values that produce minimum MSE
    '''
    # Student implementation here
    pass

## Exercise: Adaptive g-h Filter
Implement an adaptive g-h filter where g and h values change based on the residual (difference between prediction and measurement):

In [ ]:
def adaptive_g_h_filter(data, x0, dx, g_base, h_base, dt=1., adaptation_rate=0.1):
    '''
    Implements an adaptive g-h filter where g and h values
    adapt based on the magnitude of the residual.

    When residuals are large, increase g to follow measurements more closely.
    When residuals are small, decrease g to smooth more aggressively.
    '''
    # Student implementation here
    pass

In [ ]:
# Test adaptive filter with synthetic data
x0=0; dx=1; count=30; noise_factor=1
rng = np.random.RandomState(42)
measurements = gen_data(x0,dx, count, noise_factor)
# adaptive_estimates = adaptive_g_h_filter(data=measurements, x0=0., dx=1., g_base=0.5, h_base=0.5, dt=1., adaptation_rate=0.1)
# plot_g_h_results(measurements, adaptive_estimates)


## Part D. Reflection  

Answer in a Markdown cell (3–5 sentences):  
- What happens when g is too large? Too small?  
- What happens when h is too large? Too small?  
- In what situations might an α–β filter be *“good enough”*?  
- When would you need a more powerful approach (like the Kalman filter)?  

## Your answer here

## Application


Code below generates data for a ball's position as captured by a camera.

It is based on a talk the book's author gave in Berlin.

In [ ]:
def gen_ball_vision_data(sp=0.1):
    '''
    Simulate 1 sec long ball drop captured at 100 Hz from a camera
    In:
        'sp': Sigma for position noise
    Out:
        'Xr', 'Yr', 'Zr': True (un-noisy) XYZ coordinates of the ball.
        'Xm', 'Ym', 'Zm': Measured (noisy) XYZ coordinates of the ball.
    '''

    rng = np.random.RandomState(42)

    Hz = 100.0 # Frequency of Vision System
    dt = 1.0/Hz
    T = 1.0 # s measuremnt time
    m = int(T/dt) # number of measurements

    px= 0.0 # x Position Start
    py= 0.0 # y Position Start
    pz= 1.0 # z Position Start

    vx = 10.0 # m/s Velocity at the beginning
    vy = 0.0 # m/s Velocity
    vz = 0.0 # m/s Velocity

    c = 0.1 # Drag Resistance Coefficient
    d = 0.9 # Damping

    Xr=[]
    Yr=[]
    Zr=[]
    for i in range(int(m)):
        accx = -c*vx**2  # Drag Resistance

        vx += accx*dt
        px += vx*dt

        accz = -9.806 + c*vz**2 # Gravitation + Drag
        vz += accz*dt
        pz += vz*dt

        if pz<0.01:
            vz=-vz*d
            pz+=0.02
        if vx<0.1:
            accx=0.0
            accz=0.0

        Xr.append(px)
        Yr.append(py)
        Zr.append(pz)

    #arrayify
    Xr = np.asarray(Xr)
    Yr = np.asarray(Yr)
    Zr = np.asarray(Zr)
    #Add noise
    Xm = Xr + sp * (rng.randn(m))
    Ym = Yr + sp * (rng.randn(m))
    Zm = Zr + sp * (rng.randn(m))

    return Xr, Yr, Zr, Xm, Ym, Zm

In [ ]:
Xr, Yr, Zr, Xm, Ym, Zm = gen_ball_vision_data(0.2)


Plot the data:

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure(figsize=(12,6))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(Xm, Ym, Zm)
ax.scatter(Xr, Yr, Zr)

ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
plt.title('Ball Trajectory observed from Computer Vision System (with Noise)')

#ax.w_xaxis.set_pane_color((1.0, 1.0, 1.0, 1.0))

# Axis equal
max_range = np.array([Xm.max()-Xm.min(), Ym.max()-Ym.min(), Zm.max()-Zm.min()]).max() / 3.0
mean_x = Xm.mean()
mean_y = Ym.mean()
mean_z = Zm.mean()
ax.set_xlim(mean_x - max_range, mean_x + max_range)
ax.set_ylim(mean_y - max_range, mean_y + max_range)
ax.set_zlim(mean_z - max_range, mean_z + max_range)
# plt.savefig('BallTrajectory-Computervision.png', dpi=150, bbox_inches='tight')

Experiment below and see if you can figure out `g` and `h` values that can improve over the measurements.

Use the `tuner()` and `plot_g_h_results()` in your work.

Enter your comments in the cell below that describes your choice of `g` and `h` and your reasoning.

In [ ]:
#test your code here


In [ ]:
#test your code here


 **Submission checklist:**  
- [ ] All code cells run without errors.  
- [ ] MSE function is implemented and vectorized.  
- [ ] Plots are included for different (g, h) values.  
- [ ] Heatmap for (g, h)
- [ ] Adaptive filter
- [ ] Written reflection is complete.  